# Analyzing the results

The analysis pipeline of the various trajectories saves the files in .csv format. The results can be read and aggregated using pandas.

The selection of uncorrelated frames must happen after all observations have been loaded.

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src/'))
from pathlib import Path

In [12]:
import pandas as pd
import numpy as np
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from itertools import chain

In [13]:
proj = Path().absolute().parent
path_data = proj / 'data/'
input_dir = path_data / '03-processed/COM_twn'

In [14]:
# Functions needed to load data
def get_details(input_dir):
    """Reobtain catenane properties from directory name"""
    input_dir = Path(input_dir)
    els = re.split('[MmnN"Malt"]', input_dir.name)
    m = int(els[1])
    n = int(els[2])
    m_a = int(els[-1])
    return m, n, m_a


def load_obs(simdir):
    """Load observations (global)"""
    fname = simdir.name +'_global_properties.csv'
    df = pd.read_csv(simdir/fname)
    m, n, m_a = get_details(simdir.name)
    obs_cols = df.columns.to_list()
    df['M'] = m
    df['n'] = n
    df['Malt'] = m_a
    cols = ['M','n','Malt'] + obs_cols
    return df[cols]

In [17]:
#obs = pd.concat([load_obs(system) for system in input_dir.glob('M*')])
l = []
for sys in input_dir.glob('M*Ma*'):
    try:
        print(sys)
        l.append(load_obs(sys))
    except Exception as e:
        print(f"Error {e} in dir {sys} - SKIPPING")
        continue
obs = pd.concat(l,ignore_index=True)
obs['resc_rg2'] = obs['rg2'] / (obs['n']**2) * np.pi * np.pi #  rescale rg by the nominal rings' diameter

/Users/luca.tubiana/Projects/ring-o-rings/github-code/data/03-processed/COM_twn/M20n48Malt0
/Users/luca.tubiana/Projects/ring-o-rings/github-code/data/03-processed/COM_twn/M20n48Malt8
/Users/luca.tubiana/Projects/ring-o-rings/github-code/data/03-processed/COM_twn/M20n48Malt6
/Users/luca.tubiana/Projects/ring-o-rings/github-code/data/03-processed/COM_twn/M20n48Malt4
/Users/luca.tubiana/Projects/ring-o-rings/github-code/data/03-processed/COM_twn/M20n48Malt2
/Users/luca.tubiana/Projects/ring-o-rings/github-code/data/03-processed/COM_twn/M20n48Malt10


# Notation
As specified in notebook 1.0, the notation is slightly different from the one used in our manuscript. Specifically
- $n$ -> $M$
- $m$ - > $n$
- $n_{tw}$ -> $M_{alt}$

In the following each combination of `M`, `n` and `Malt` identify a trajectory and each line correspond to a frame in the trajectory. 
- `frame` refers to the frame of the snapshot in units of $10^6 \tau_{int} = 10^6 * 0.0124 \tau_{LJ}$,
- `rg2` is the squared radius of gyration of the catenane,
- `resc_rg2` is `rg2` rescaled by the nominal diameter of the elementary ring,
- `twist` is the total twist of the catenane,
- `writhe` is the total writhe of the catenane,
- `C_norm_d` is the correlation between norms separated by $i$ bonds, $|\mathbf{\hat{N}}_k\cdot\mathbf{\hat{N}}_{k+d}|$,
- `C_tan_d` is $\mathbf{\hat{T}}_k\cdot\mathbf{\hat{T}}_{k+d}$ (unused)

In [20]:
obs.head()

,M,n,Malt,frame,rg2,twist,writhe,contour_length,C_tan_1,C_tan_2,...,C_norm_12,C_norm_13,C_norm_14,C_norm_15,C_norm_16,C_norm_17,C_norm_18,C_norm_19,collinearity,resc_rg2
0,20,48,0,0,1332.862617,0.016304,-0.016304,244.553425,117.462991,102.997355,...,0.534982,0.562941,0.297771,0.422083,0.416961,0.463214,0.568393,0.392811,0.406302,5.709560
1,20,48,0,1,332.108871,-0.279788,0.279788,181.769897,33.703859,15.375862,...,0.476488,0.515660,0.434910,0.546840,0.548871,0.531834,0.525296,0.363836,3.023353,1.422649
2,20,48,0,2,388.052330,0.313941,-0.313941,196.216538,35.955247,10.230299,...,0.581350,0.486140,0.372091,0.655807,0.465185,0.623940,0.451195,0.365103,2.718519,1.662293
3,20,48,0,3,415.750055,-0.396757,0.396757,197.853775,33.436351,21.341401,...,0.598448,0.530427,0.439267,0.500243,0.579881,0.418992,0.478817,0.437399,2.354529,1.780941
4,20,48,0,4,419.101438,0.151298,-0.151298,198.273064,18.481536,15.774130,...,0.569657,0.463486,0.526712,0.482722,0.311605,0.562539,0.537913,0.424217,2.213203,1.795297


# Example: Twist + Writhe
In the example below we sum twist and writhe and compute their integer and fractional part, demonstrating that $Tw+Wr\in \mathbb{Z}$ for every conformation.


In [19]:
tw_p_wr = obs.twist + obs.writhe

In [21]:
frac, intg = np.modf(tw_p_wr) # return fractional and integer parts

Since the fractional part can often be 0.999999, we need to rescale it so that is included between $\left[-0.5,0.5\right]$

In [25]:
frac[frac>0.5] = 1.-frac[frac>0.5]
frac[frac<-0.5] = 1.+frac[frac<-0.5]

The results clearly demonstrate that $Tw+Wr\in \mathbb{Z}$ aside for numerical errors:

In [34]:
print(f"""
Maximum deviation: {np.abs(frac).max():.2e} 
Average deviation: {frac.mean():.2e} 
Standard deviation: {frac.std():.2e}
""")


Maximum deviation: 5.76e-12 
Average deviation: 1.41e-14 
Standard deviation: 1.40e-13

